In [1]:
from pymongo import MongoClient
import pandas as pd
import json as js
import folium

In [2]:
# It is a problem to have many companies with multiple office locations. I (we) solved that
# using "unwind", and "match" to link the whole query

def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, coll = connectCollection('companies','companies')
pipeline = [
    { "$unwind": "$offices"},
    {"$match":{ "$and": [  {"founded_year":{"$gt":2000}},{"funding_rounds.raised_amount":{"$gt":1000000}}] }}
    
 ]
results = list(coll.aggregate(pipeline))

In [3]:
len(results)

3278

In [4]:
# i had problems of unknown nature. I had a beautiful way to extract long and latitude in a 
# a list to implement in folium, but now that is gone and i had to make up a patch 
location_raw=[]
name=[]
lon=[]
lat=[]

for element in results:
    name.append(element["name"])
    for coord in element["offices"].items():
        
        #print(coord)
        
        if coord[0]=="longitude":
            lon.append(coord[1])
        elif coord[0]=="latitude":
            lat.append(coord[1])

In [5]:
print(len(name),len(lat),len(lon))

3278 3278 3278


In [6]:
l={"name":name, "longitude":lon, "latitude":lat}
data_locations=pd.DataFrame(data=l)

In [7]:
data_locations

,name,longitude,latitude
0,Wetpaint,-122.333253,47.603122
1,Wetpaint,-73.996431,40.723731
2,Digg,-122.394523,37.764726
3,Geni,-118.393064,34.090368
4,StumbleUpon,-122.419204,37.775196
...,...,...,...
3273,Treehouse,NaN,NaN
3274,Cameron Health,-117.607717,33.448010
3275,Cantimer,-122.181753,37.484616
3276,Celestial Semiconductor,-121.929170,37.386417


In [10]:
data_clean=data_locations.dropna()

In [11]:
data_clean

,name,longitude,latitude
0,Wetpaint,-122.333253,47.603122
1,Wetpaint,-73.996431,40.723731
2,Digg,-122.394523,37.764726
3,Geni,-118.393064,34.090368
4,StumbleUpon,-122.419204,37.775196
...,...,...,...
3270,Advanced Power Projects,-121.917842,37.525246
3271,Luminus Devices,-71.278022,42.528635
3274,Cameron Health,-117.607717,33.448010
3275,Cantimer,-122.181753,37.484616


In [12]:
lat=list(data_clean["latitude"])
long=list(data_clean["longitude"])
name=list(data_clean["name"])

In [13]:
world_map=folium.Map(location=[52.5200,13.4050], zoom_start=12)

In [14]:
for i in range(len(lat)):
    folium.Marker([lat[i],long[i]],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(world_map)

In [17]:
airports.ipynb/airports()

NameError: name 'airports' is not defined

In [15]:
world_map

In [34]:
l={"name":name, "location":location_list}
data_locations=pd.DataFrame(data=l)

In [35]:
data_locations

,name,location
0,Facebook,"{'type': 'Point', 'coordinates': [-122.151801,..."
1,Facebook,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
2,Facebook,"{'type': 'Point', 'coordinates': [-73.9792469,..."
3,Twitter,"{'type': 'Point', 'coordinates': [-122.4169244..."
4,AddThis,"{'type': 'Point', 'coordinates': [-77.245195, ..."
...,...,...
535,FleetMatics,"{'type': 'Point', 'coordinates': [None, None]}"
536,FleetMatics,"{'type': 'Point', 'coordinates': [None, None]}"
537,800APP,"{'type': 'Point', 'coordinates': [None, None]}"
538,InfoLogix,"{'type': 'Point', 'coordinates': [-75.1069232,..."


In [43]:
world_map=folium.Map(location=[52.5200,13.4050], zoom_start=12)
for company in data_locations:
world_map

In [50]:
#si quiero añadir información a la colección $set (para el geoindex)
for place in rest:
    value = {"$set": {'location':getLocation(place)}}
    coll.update_one(place,value)

In [51]:
# eliminar las antiguas coordenadas
for place in rest:
    value = {"$unset": {'offices.latitude':''}}
    value = {"$unset": {'offices.longitude':''}}
    coll.update_one(place,value)
    # <collection>.update_one(<document>,<value>)

In [54]:
'''
prefered_city = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-73.9712,40.7831]
       },
       "$maxDistance":10000
      }
     }
    }
)
prefered_city=list(prefered_city)
'''

'\nprefered_city = coll.find(\n    {"location":\n     {"$near":\n      {"$geometry":\n       {"type":"Point",\n        "coordinates":[-73.9712,40.7831]\n       },\n       "$maxDistance":10000\n      }\n     }\n    }\n)\nprefered_city=list(prefered_city)\n'

In [58]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)
map_city
'''
for wendy in manhattanWendys:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    
for wendy in manhattanWendys[:5]:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='blue'), 
                   ).add_to(map_city)
'''

"\nfor wendy in manhattanWendys:\n    folium.Marker(wendy['location']['coordinates'][::-1],\n                    radius=2,\n                    icon=folium.Icon(icon='cloud',color='red'), \n                   ).add_to(map_city)\n    \nfor wendy in manhattanWendys[:5]:\n    folium.Marker(wendy['location']['coordinates'][::-1],\n                    radius=2,\n                    icon=folium.Icon(icon='cloud',color='blue'), \n                   ).add_to(map_city)\n"